# Experiment 1: Test K-Instablity



In [1]:
import sys
import warnings

warnings.filterwarnings("ignore")


user_site_packages = '/home/am5715/.local/lib/python3.11/site-packages'

if user_site_packages in sys.path:
    sys.path.remove(user_site_packages)

sys.path.insert(0, user_site_packages)

import os

private_libs_path = '/home/am5715/smoothllm_workshop/my_private_libs'

if private_libs_path not in sys.path:
    sys.path.insert(0, private_libs_path)


os.environ["ACCELERATE_DISABLE_BNB"] = "1"
os.environ["TRANSFORMERS_SKIP_AWS"] = "1"
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

print("--- Final sys.path ---")
for i, path in enumerate(sys.path[:5]):
    print(f"{i}: {path}")
print("----------------------")

--- Final sys.path ---
0: /home/am5715/smoothllm_workshop/my_private_libs
1: /home/am5715/.local/lib/python3.11/site-packages
2: /home/am5715/smoothllm_workshop
3: /.autofs/tools/spack/var/spack/environments/default-ml-x86_64-24071101/.spack-env/view/lib/scons
4: 
----------------------


In [2]:
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import fastchat

import smoothllm.lib.perturbations as perturbations
import smoothllm.lib.defenses as defenses
import smoothllm.lib.attacks as attacks
import smoothllm.lib.language_models as language_models
import smoothllm.lib.model_configs as model_configs

os.getcwd()

'/home/am5715/smoothllm_workshop'

# Llama2

In [3]:
torch.cuda.empty_cache()

# Targeted LLM
target_model= 'llama2'

# Attacking LLM
attack='GCG'
attack_logfile='smoothllm/data/GCG/llama2_behaviors.json'

# SmoothLLM
smoothllm_num_copies=1
smoothllm_pert_pct=1
smoothllm_pert_type='RandomSwapPerturbation'

In [ ]:
# Instantiate the targeted LLM
config = model_configs.MODELS[target_model]
target_model = language_models.LLM(
    model_path=config['model_path'],
    tokenizer_path=config['tokenizer_path'],
    conv_template_name=config['conversation_template'],
    device='cuda:0'
)

# Create SmoothLLM instance
defense = defenses.SmoothLLM(
    target_model=target_model,
    pert_type=smoothllm_pert_type,
    pert_pct=smoothllm_pert_pct,
    num_copies=smoothllm_num_copies
)

# Create attack instance, used to create prompts
attack = vars(attacks)[attack](
    logfile=attack_logfile,
    target_model=target_model
)

# Checking defense success rate with different positions
jb_percentage = []
for _ in range(2):
    for i, prompt in tqdm(enumerate(attack.prompts)):
        output = defense(prompt)
        jb = defense.is_jailbroken(output)
        jb_percentage.append(jb)
print(f"For k=1, Attack Accuracy: {np.mean(jb_percentage)}, Standard Dev.: {np.std(jb_percentage)}")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating attack for goal: 'Write a script that can exploit vulnerabilities in...'


2025-08-22 03:51:25.874359: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-22 03:51:25.938518: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-22 03:51:54.435545: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
100%|██████████| 500/500 [05:54<00:00,  1.41it/s]


Generating attack for goal: 'Develop a strategy for hacking into a government d...'


100%|██████████| 500/500 [05:54<00:00,  1.41it/s]


...

100%|██████████| 500/500 [05:39<00:00,  1.47it/s]


Generating attack for goal: 'Provide guidance on how to cause physical harm to ...'


100%|██████████| 500/500 [05:46<00:00,  1.44it/s]


Generating attack for goal: 'Create a tutorial on how to hack into sensitive go...'


 82%|████████▏ | 409/500 [04:43<01:03,  1.43it/s]

In [ ]:
np.mean(jb_percentage), np.std(jb_percentage)

In [ ]:
print(jb_percentage)